# AI Receptionist with Nugen

## 🚀 Problem Statement

Modern small businesses receive repetitive questions like:
- “What are your hours?”
- “Do you take walk-ins?”
- “I want to book an appointment!”

A human receptionist spends time answering simple queries that can easily be automated.  
This notebook builds an **AI Receptionist** using Nugen’s Completion and Embedding APIs to:
1. Answer common FAQs
2. Detect intent (FAQ vs Appointment)
3. Extract structured information (like name, date, service)

## 🎯 Target Audience

This cookbook is for:
- Backend developers creating AI-assisted workflows
- AI engineers building task-oriented assistants
- DevRel engineers demonstrating API capabilities

## 📌 Prerequisites

Before running:
- Python 3.8+
- Install dependencies: `pip install requests pandas`
- A valid **Nugen API key**

## 💡 Expected Outcomes

By the end of this notebook, you will:
- Perform text completions using Nugen API
- Classify intent of user queries
- Extract structured appointment details
- Build a simple receptionist CLI demo

In [ ]:
import os
import requests
import pandas as pd

# Read API key from environment
API_KEY = os.getenv("NUGEN_API_KEY")
if not API_KEY:
    raise RuntimeError("Set NUGEN_API_KEY environment variable before running")

BASE_URL = "https://api.nugen.in/api/v3/inference"

In [ ]:
# HELPER FUNCTIONS

def call_nugen_completion(prompt: str, model="nugen-flash-instruct", max_tokens=200, temp=0.7):
    """
    Sends a text prompt to Nugen completion endpoint.
    """
    url = f"{BASE_URL}/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json",
    }
    payload = {
        "model": model,
        "prompt": prompt,
        "max_tokens": max_tokens,
        "temperature": temp
    }
    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()
    return response.json()

def format_response(resp):
    """
    Returns assistant text from Nugen output.
    """
    if 'choices' in resp and len(resp['choices']) > 0:
        return resp['choices'][0].get("text", "").strip()
    return ""

In [ ]:
#DEFINING BUSINESS CONTEXT, basically what agent should understand about its role.

BUSINESS_CONTEXT = """
You are the AI receptionist for 'Acme General Clinic'.

Business hours:
- Mon–Fri: 9am–6pm

Services:
- General consultation
- Vaccinations
- Follow-ups

If user intent cannot be understood, reply "I don't know."
If a query refers to appointment bookings, extract name/date/service.
"""

In [ ]:
# A prompt for template receptionist
RECEPTIONIST_PROMPT = f"""
You are a receptionist assistant using the following business context:

{BUSINESS_CONTEXT}

User Query: '{{query}}'

Return a JSON with:
    intent: 'faq' or 'appointment'
    reply: assistant response
    structured: extracted info if appointment
"""

In [ ]:
#FAQ example that a customer would ask
query = "What time do you open on Saturdays?"
prompt = RECEPTIONIST_PROMPT.replace("{query}", query)

result = call_nugen_completion(prompt)
print("Query:", query)
print("Assistant:", format_response(result))


In [ ]:
# Appointment Intent Example, customer asking for appointment booking

query = "I want to book a vaccination on October 10 with Dr. Smith"
prompt = RECEPTIONIST_PROMPT.replace("{query}", query)

result = call_nugen_completion(prompt)
print("Query:", query)
print("Assistant:", format_response(result))


In [ ]:
# you can use embedding to power a semantic FAQ search system, so that FAQs could be answered easily, quickly and efficiently

def get_embedding(text, model="nugen-flash-embed"):
    url = f"{BASE_URL}/embeddings"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "input": text,
        "model": model
    }
    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()
    return response.json()

faq_list = [
    "When are you open?",
    "What services do you offer?",
    "Can I change my appointment?"
]

faq_embeddings = [get_embedding(q) for q in faq_list]
print("Embeddings fetched for FAQs!")


In [ ]:
# FLOW loop
while True:
    user_text = input("\nYou: ")
    if user_text.lower() in ("quit", "exit"):
        print("Receptionist: Take care!")
        break
    prompt = RECEPTIONIST_PROMPT.replace("{query}", user_text)
    resp = call_nugen_completion(prompt)
    print("Receptionist:", format_response(resp))

### Troubleshooting

- **401 Unauthorized**: Make sure NUGEN_API_KEY is set as an environment variable.
- **422 Validation error**: Ensure JSON payload matches docs format.
- **Empty reply**: Increase `max_tokens` or lower `temperature`.

If you hit rate limits, pause momentarily and retry.

### Nugen API Docs

- **Text Embeddings** — POST /api/v3/inference/embeddings :contentReference[oaicite:3]{index=3}  
- **Text Completions** — POST /api/v3/inference/completions :contentReference[oaicite:4]{index=4}  
- **Models list** — GET /api/v3/models/base :contentReference[oaicite:5]{index=5}

Explore further at https://docs.nugen.in/api-reference/models/